In [1]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

### LLM에 도구 바인딩 (Binding Tools) , JsonOutputToolsParser , Execution(execute_tool_calls)

`bind_tools` + `JsonOutputToolsParser` + `execute_tool_calls` <br>

- llm_with_tools : 도구를 바인딩한 모델 (llm + 도구들). llm_with_tools는 도구를 가지고 있다.
- JsonOutputToolsParser : llm_with_tools를 호출했을 때 나온 결과를 파싱하는 파서 (사용할 도구 이름, 전달 되는 인자와 같은 판단을 보기 좋게 출력)
- execute_tool_calls : 도구 호출 결과(tool_call_result)를 인자로 받아서 도구를 직접 찾아서 실행하는 함수.

In [2]:
from bs4 import BeautifulSoup
from langchain.agents import tool
import requests
import re

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser

In [4]:
@tool
def get_word_length(word: str) -> int:          # 단어의 길이를 리턴하는 도구
    """Returns the length of a word."""

    return len(word)

In [5]:
@tool
def add_function(a: float, b: float) -> float:  # 두 수의 합을 리턴하는 도구
    """Adds two numbers together."""
    
    return a + b

In [6]:
@tool
def naver_news_crawl(news_url: str) -> str:                 # 네이버 뉴스기사를 크롤링하여 본문 내용을 리턴하는 도구
    """Crawls a 네이버(naver.com) news article and returns the body content."""

    response = requests.get(news_url)                       # HTTP GET 요청

    if response.status_code == 200:                         # 응답 200. 성공
        soup = BeautifulSoup(response.text, 'html.parser')

        title = soup.find('h2', id='title_area').get_text()
        content = soup.find('div', id='contents').get_text()    
        
        cleaned_title = re.sub(r'\n{2, }', '\n', title)
        cleaned_content = re.sub(r'\n{2, }', '\n', content)
    else:
        print(f'HTTP 요청 실패. {response.status_code}')
    
    return f'{cleaned_title}\n{cleaned_content}'

In [7]:
tools = [get_word_length, add_function, naver_news_crawl]   # 여러 개의 도구를 한꺼번에 바인딩 위해서 리스트에 넣기

In [8]:
llm = ChatOpenAI(
    api_key=key, 
    model='gpt-4o-mini', 
    temperature=0
)

llm_with_tools = llm.bind_tools(tools)                      # 도구 바인딩(llm_with_tools는 도구를 가지고 있다.)

In [9]:
# tool_call_results 파라미터 : chain 실행의 결과(어떤 도구를 사용해야 하는지와 도구에 전달 되는 인자와 같은 판단)를 받는다
def execute_tool_calls(tool_call_results):
    """
    도구 호출 결과를 실행하는 함수

    :param tool_call_results: 도구 호출 결과 리스트
    :param tools: 사용 가능한 도구 리스트
    """

    print(f'tool_call_results : {tool_call_results}')

    for tool_call_result in tool_call_results:      # chain을 호출 했을 때의 결과를 가지고 있는 tool_call_results(도구 호출 결과 리스트)를 반복해서 
                                                    # 사용할 도구의 이름과 도구에 전달되는 인자를 추출한다. 
        
        tool_name = tool_call_result['type']        # 도구의 이름 
        tool_args = tool_call_result['args']        # 도구에 전달되는 인자

        print(f'tool_name : {tool_name}')
        print(f'tool_args : {tool_args}')
        
        matching_tool = None                        

        for tool in tools:                              # tools 리스트에 정의 된 도구 이름과 일치하는 첫 번째 도구를 찾습니다.
            if tool.name == tool_name:                  # tools 리스트에 정의 된 도구 이름과 chain의 결과로 나온 사용할 도구 이름과 같은 이름이면
                matching_tool = tool                    # matching_tool 변수에 넣는다
                break
        
        if matching_tool:                               # 일치하는 도구를 찾은 경우
            result = matching_tool.invoke(tool_args)    # tool_args 를 넣어서 해당 도구를 실행한다. 여기서 직접 호출해서 얻은 결과가 최종 실행결과 이다.

            print(f'[실행도구] {tool_name} [Argument] {tool_args}')     # 실행 할 도구 이름, tool_args를 출력       
            print(f'[실행결과] {result}')                               # 실행결과 출력    
        else:                                           # 일치하는 도구를 찾지 못한 경우
            print(f'경고: {tool_name}에 해당하는 도구를 찾을 수 없습니다.')

In [10]:
# llm_with_tools으로 도구를 실행해서 나온 결과(실행할 때 사용할 도구의 이름, 도구에 전달되는 인자)를 도구 목록이 정의 된 JsonOutputToolsParser(tools=tools)에 연결
# 도구 바인딩 + 도구 파서 + 도구 호출 결과를 실행하는 함수
chain = llm_with_tools | JsonOutputToolsParser(tools=tools) | execute_tool_calls

In [11]:
chain.invoke('랭체인 이라는 단어의 글자 길이는?')

tool_call_results : [{'args': {'word': '랭체인'}, 'type': 'get_word_length'}]
tool_name : get_word_length
tool_args : {'word': '랭체인'}
[실행도구] get_word_length [Argument] {'word': '랭체인'}
[실행결과] 3


In [12]:
chain.invoke('90 + 10 = ?')

tool_call_results : [{'args': {'a': 90, 'b': 10}, 'type': 'add_function'}]
tool_name : add_function
tool_args : {'a': 90, 'b': 10}
[실행도구] add_function [Argument] {'a': 90, 'b': 10}
[실행결과] 100.0


In [13]:
chain.invoke(
    "네이버 뉴스 기사의 내용을 크롤링해주세요: https://n.news.naver.com/mnews/article/002/0002372963"
)

tool_call_results : [{'args': {'news_url': 'https://n.news.naver.com/mnews/article/002/0002372963'}, 'type': 'naver_news_crawl'}]
tool_name : naver_news_crawl
tool_args : {'news_url': 'https://n.news.naver.com/mnews/article/002/0002372963'}
[실행도구] naver_news_crawl [Argument] {'news_url': 'https://n.news.naver.com/mnews/article/002/0002372963'}
[실행결과] 국민의힘, 反탄핵 '정치투쟁' 총력…"내란은 민주당 기획"



'부정선거론' 주류화하나…권영세 "선거 부실 걱정하니 극우 딱지"헌법재판소의 윤석열 대통령 탄핵심판이 종반부로 접어들면서, 탄핵 반대를 내세운 국민의힘의 정치투쟁이 극에 달한 양상을 보이고 있다. 권영세 비상대책위원장은 "(민주당이) 선거관리 부실을 걱정하는 국민들에게 극우 딱지를 붙인다", "눈 떠보니 극우화됐다"는 등 부정선거 음모론을 설파하는 극우성향 지지층 집회를 감싸고 나섰다. 권 위원장은 또 헌재 및 탄핵심판 증인들을 맹비난하면서 "자유를 지키기 위해 치열하게 맞서 싸울 것"이라고 하는 등, 윤 대통령이 '서부지법 폭동' 전 지지자들을 호도했던 발언 내용을 연상시켜 눈길을 끌기도 했다. 권 위원장은 13일 오전 국회 비상대책위원회의 모두발언에서 "(민주당이) 선거관리 부실을 걱정하는 국민들에게 극우 딱지를 붙이고, 탄핵에 반대하면 내란동조 세력이라고 몰아 붙이고 있다"며 "많은 국민들이 '눈 떠보니 극우화됐다'며 민주당을 비웃고 있다"고 주장했다. 국민의힘에선 지난 8일 대구 탄핵반대 집회를 계기로 당 비대위원 및 수석대변인 등이 '탄핵반대 집회를 극우로 몰지 말라'는 취지로 주장해왔다. 이에 더해 당 대표까지 나서 부정선거 음모론과 계엄 정당화를 주장하고 있는 강성